# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,-24.15,64,75,1.03,CA,1707605425
1,1,ain sefra,32.7500,-0.5833,8.07,78,93,2.98,DZ,1707605425
2,2,west island,-12.1568,96.8225,27.99,78,100,5.14,CC,1707605425
3,3,grytviken,-54.2811,-36.5092,3.60,85,24,8.43,GS,1707605270
4,4,st. john's,47.5649,-52.7093,-2.96,91,20,2.06,CA,1707605315


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=700,
    frame_height=500,
    size='Humidity',
    color='City', 
    alpha=0.5
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] > 21) & 
                                (city_data_df['Max Temp'] < 27) & 
                                (city_data_df['Wind Speed'] < 4.5) & 
                                (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,mount gambier,-37.8333,140.7667,22.11,46,0,0.90,AU,1707605409
78,78,sangueya,10.7000,-14.3667,23.37,85,0,1.44,GN,1707605449
200,200,mount pleasant,32.7941,-79.8626,21.16,81,0,3.09,US,1707605488
207,207,codrington,-38.2667,141.9667,22.15,49,0,1.69,AU,1707605490
281,281,sao filipe,14.8961,-24.4956,24.52,63,0,3.21,CV,1707605511


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,iqaluit,CA,63.7506,-68.5145,64,
1,ain sefra,DZ,32.7500,-0.5833,78,
2,west island,CC,-12.1568,96.8225,78,
3,grytviken,GS,-54.2811,-36.5092,85,
4,st. john's,CA,47.5649,-52.7093,91,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "limit": 1,  # Limit to one result per city
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] = f"proximity:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
iqaluit - nearest hotel: No hotel found
ain sefra - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
st. john's - nearest hotel: No hotel found
merizo village - nearest hotel: No hotel found
susuman - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
birnin kudu - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
mariental - nearest hotel: No hotel found
morelos - nearest hotel: No hotel found
bambari - nearest hotel: No hotel found
ouadda - nearest hotel: No hotel found
hadibu - nearest hotel: Gut Nustrow
longyearbyen - nearest hotel: No hotel found
walvis bay - nearest hotel: No hotel found
sao joao evangelista - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
mount gambier - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
faaborg - nearest hotel: No hotel found
bil

,City,Country,Lat,Lng,Humidity,Hotel Name
0,iqaluit,CA,63.7506,-68.5145,64,No hotel found
1,ain sefra,DZ,32.7500,-0.5833,78,No hotel found
2,west island,CC,-12.1568,96.8225,78,No hotel found
3,grytviken,GS,-54.2811,-36.5092,85,No hotel found
4,st. john's,CA,47.5649,-52.7093,91,No hotel found
...,...,...,...,...,...,...
537,illizi,DZ,26.4833,8.4667,9,No hotel found
538,rakops,BW,-21.0167,24.3333,51,No hotel found
539,zhangjiajie,CN,29.1294,110.4783,85,No hotel found
540,savannah,US,32.0835,-81.0998,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot

filtered_hotel_df = hotel_df[hotel_df['Hotel Name'] != 'No hotel found']

map_plot = filtered_hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=700,
    frame_height=500,
    size='Humidity',
    color='City',
    hover_cols=['City', 'Country', 'Hotel Name'], 
    alpha=0.5
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)